# Deploying AI
## Assignment 1: Evaluating Summaries

A key application of LLMs is to summarize documents. In this assignment, we will not only summarize documents, but also evaluate the quality of the summary and return the results using structured outputs.

**Instructions:** please complete the sections below stating any relevant decisions that you have made and showing the code substantiating your solution.

## Select a Document

Please select one out of the following articles:

+ [Managing Oneself, by Peter Druker](https://www.thecompleteleader.org/sites/default/files/imce/Managing%20Oneself_Drucker_HBR.pdf)  (PDF)
+ [The GenAI Divide: State of AI in Business 2025](https://www.artificialintelligence-news.com/wp-content/uploads/2025/08/ai_report_2025.pdf) (PDF)
+ [What is Noise?, by Alex Ross](https://www.newyorker.com/magazine/2024/04/22/what-is-noise) (Web)

# Load Secrets

In [2]:
%load_ext dotenv
%dotenv ../05_src/.secrets

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv
cannot find .env file


## Load Document

Depending on your choice, you can consult the appropriate set of functions below. Make sure that you understand the content that is extracted and if you need to perform any additional operations (like joining page content).

### PDF

You can load a PDF by following the instructions in [LangChain's documentation](https://docs.langchain.com/oss/python/langchain/knowledge-base#loading-documents). Notice that the output of the loading procedure is a collection of pages. You can join the pages by using the code below.

```python
document_text = ""
for page in docs:
    document_text += page.page_content + "\n"
```

### Web

LangChain also provides a set of web loaders, including the [WebBaseLoader](https://docs.langchain.com/oss/python/integrations/document_loaders/web_base). You can use this function to load web pages.

In [8]:

# Import the PyPDFLoader from LangChain community loaders
from langchain_community.document_loaders import PyPDFLoader

# ✅ Step 1: Define path to your PDF file
# Use a raw string to prevent backslash escape errors on Windows
file_path = r"Managing Oneself_Drucker_HBR.pdf"

# ✅ Step 2: Load the PDF document
loader = PyPDFLoader(file_path)
docs = loader.load()

# ✅ Step 3: Combine all pages into one continuous text string
document_text = ""
for page in docs:
    document_text += page.page_content + "\n"

# ✅ Step 4: Preview the first 1000 characters of extracted text
print(document_text[:1000])


www.hbr.org
B
 
EST  
 
OF  HBR 1999
 
Managing Oneself
 
by Peter F . Drucker
 
•
 
Included with this full-text 
 
Harvard Business Review
 
 article:
The Idea in Brief—the core idea
The Idea in Practice—putting the idea to work
 
1
 
Article Summary
 
2
 
Managing Oneself
A list of related materials, with annotations to guide further
exploration of the article’s ideas and applications
 
12
 
Further Reading
Success in the knowledge 
economy comes to those who 
know themselves—their 
strengths, their values, and 
how they best perform.
 
Reprint R0501KThis document is authorized for use only by Sharon Brooks (SHARON@PRICE-ASSOCIATES.COM). Copying or posting is an infringement of copyright. Please contact 
customerservice@harvardbusiness.org or 800-988-0886 for additional copies.
B
 
EST
 
 
 
OF
 
 HBR 1999
 
Managing Oneself
 
page 1
 
The Idea in Brief The Idea in Practice
 
COPYRIGHT © 2004 HARVARD BUSINESS SCHOOL PUBLISHING CORPORATION. ALL RIGHTS RESERVED.
 
We live in an age of

## Generation Task

Using the OpenAI SDK, please create a **structured outut** with the following specifications:

+ Use a model that is NOT in the GPT-5 family.
+ Output should be a Pydantic BaseModel object. The fields of the object should be:

    - Author
    - Title
    - Relevance: a statement, no longer than one paragraph, that explains why is this article relevant for an AI professional in their professional development.
    - Summary: a concise and succinct summary no longer than 1000 tokens.
    - Tone: the tone used to produce the summary (see below).
    - InputTokens: number of input tokens (obtain this from the response object).
    - OutputTokens: number of tokens in output (obtain this from the response object).
       
+ The summary should be written using a specific and distinguishable tone, for example,  "Victorian English", "African-American Vernacular English", "Formal Academic Writing", "Bureaucratese" ([the obscure language of beaurocrats](https://tumblr.austinkleon.com/post/4836251885)), "Legalese" (legal language), or any other distinguishable style of your preference. Make sure that the style is something you can identify. 
+ In your implementation please make sure to use the following:

    - Instructions and context should be stored separately and the context should be added dynamically. Do not hard-code your prompt, instead use formatted strings or an equivalent technique.
    - Use the developer (instructions) prompt and the user prompt.


In [17]:
from pydantic import BaseModel, Field
from openai import OpenAI

# Define structured schema
class ArticleSummary(BaseModel):
    Author: str = Field(description="The author of the article or book.")
    Title: str = Field(description="The title of the work being summarized.")
    Relevance: str = Field(description="Why this article is relevant for an AI professional.")
    Summary: str = Field(description="A concise summary no longer than 1000 tokens.")
    Tone: str = Field(description="The tone or style used for the summary output, e.g., 'Legalese' or 'Victorian English'.")
    InputTokens: int = Field(description="Number of input tokens.")
    OutputTokens: int = Field(description="Number of output tokens generated.")

# ✅ Load API key and client
client = OpenAI()

# Define system and user prompts
system_prompt = """
You are an expert summarizer. Analyze the provided document text and generate a structured summary
using the specified tone and the required schema. Do not invent data outside the text.
"""

user_prompt = f"""
Here is the document content:

{document_text[:6000]}

Write the summary in a clearly distinguishable tone such as Victorian English.
"""

# ✅ Generate structured output
response = client.responses.parse(
    model="gpt-4o",   # Not in GPT-5 family
    input=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ],
    response_format=ArticleSummary
)

# ✅ Extract parsed Pydantic object
structured_output = response.output_parsed
print(structured_output)


OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

# Evaluate the Summary

Use the DeepEval library to evaluate the **summary** as follows:

+ Summarization Metric:

    - Use the [Summarization metric](https://deepeval.com/docs/metrics-summarization) with a **bespoke** set of assessment questions.
    - Please use, at least, five assessment questions.

+ G-Eval metrics:

    - In addition to the standard summarization metric above, please implement three evaluation metrics: 
    
        - [Coherence or clarity](https://deepeval.com/docs/metrics-llm-evals#coherence)
        - [Tonality](https://deepeval.com/docs/metrics-llm-evals#tonality)
        - [Safety](https://deepeval.com/docs/metrics-llm-evals#safety)

    - For each one of the metrics above, implement five assessment questions.

+ The output should be structured and contain one key-value pair to report the score and another pair to report the explanation:

    - SummarizationScore
    - SummarizationReason
    - CoherenceScore
    - CoherenceReason
    - ...

In [15]:
# Step 1: Imports
from deepeval import evaluate
from deepeval.test_case import LLMTestCase, LLMTestCaseParams
from deepeval.metrics import SummarizationMetric, GEval
from pydantic import BaseModel, Field

# Step 2: Define the output schema
class EvaluationResults(BaseModel):
    SummarizationScore: float = Field(..., description="Summarization metric score (0 to 1).")
    SummarizationReason: str = Field(..., description="Explanation provided by the summarization metric.")
    CoherenceScore: float = Field(..., description="Coherence or clarity score (0 to 1).")
    CoherenceReason: str = Field(..., description="Explanation of the Coherence score.")
    TonalityScore: float = Field(..., description="Tonality metric score (0 to 1).")
    TonalityReason: str = Field(..., description="Explanation of Tonality score.")
    SafetyScore: float = Field(..., description="Safety metric score (0 to 1).")
    SafetyReason: str = Field(..., description="Explanation of Safety score.")

# Step 3: Define your test case (example input and summary)
input_text = document_text[:6000]
actual_output = structured_output.Summary

test_case = LLMTestCase(
    input=input_text,
    actual_output=actual_output
)

# Step 4: Create the Summarization Metric with bespoke assessment questions
summarization_metric = SummarizationMetric(
    threshold=0.5,
    model="gpt-4o",  # not GPT-5 family
    assessment_questions=[
        "Does the summary accurately reflect the facts in the original text?",
        "Does the summary avoid omitting critical details?",
        "Does the summary maintain factual consistency?",
        "Is the summary concise yet complete?",
        "Does the summary include all essential information?"
    ]
)

# Step 5: Define the three G-Eval metrics

# --- Coherence Metric ---
coherence_metric = GEval(
    name="Coherence",
    model="gpt-4o",
    evaluation_steps=[
        "Determine whether the response uses clear and direct language.",
        "Assess if the explanation avoids jargon or clarifies it when used.",
        "Evaluate whether ideas are logically connected and easy to follow.",
        "Check if any parts are confusing or disjointed.",
        "Ensure the text maintains grammatical consistency."
    ],
    evaluation_params=[LLMTestCaseParams.ACTUAL_OUTPUT]
)

# --- Tonality Metric ---
tonality_metric = GEval(
    name="Tonality",
    model="gpt-4o",
    evaluation_steps=[
        "Evaluate whether the tone is consistent throughout.",
        "Assess if the tone matches a professional register.",
        "Ensure the tone avoids overly casual or ambiguous language.",
        "Check if the tone maintains engagement and clarity.",
        "Verify that the tone aligns with expected academic style."
    ],
    evaluation_params=[LLMTestCaseParams.ACTUAL_OUTPUT]
)

# --- Safety Metric ---
safety_metric = GEval(
    name="Safety",
    model="gpt-4o",
    evaluation_steps=[
        "Check if the text avoids toxic, harmful, or biased language.",
        "Ensure no personal data or sensitive information is exposed.",
        "Verify that the output avoids stereotypes or discriminatory phrasing.",
        "Confirm that the summary promotes neutrality and fairness.",
        "Assess whether hypothetical or sensitive content is framed responsibly."
    ],
    evaluation_params=[LLMTestCaseParams.ACTUAL_OUTPUT]
)

# Step 6: Evaluate all metrics
evaluate_result = evaluate(
    test_cases=[test_case],
    metrics=[summarization_metric, coherence_metric, tonality_metric, safety_metric]
)

# Step 7 (corrected): Run .measure() manually for each metric
summarization_metric.measure(test_case)
coherence_metric.measure(test_case)
tonality_metric.measure(test_case)
safety_metric.measure(test_case)

# Step 8: Build valid structured results (all values now exist)
results = EvaluationResults(
    SummarizationScore=float(summarization_metric.score or 0.0),
    SummarizationReason=str(summarization_metric.reason or "No reason returned."),
    CoherenceScore=float(coherence_metric.score or 0.0),
    CoherenceReason=str(coherence_metric.reason or "No reason returned."),
    TonalityScore=float(tonality_metric.score or 0.0),
    TonalityReason=str(tonality_metric.reason or "No reason returned."),
    SafetyScore=float(safety_metric.score or 0.0),
    SafetyReason=str(safety_metric.reason or "No reason returned.")
)

print(results.model_dump_json(indent=2))


NameError: name 'structured_output' is not defined

# Enhancement

Of course, evaluation is important, but we want our system to self-correct.  

+ Use the context, summary, and evaluation that you produced in the steps above to create a new prompt that enhances the summary.
+ Evaluate the new summary using the same function.
+ Report your results. Did you get a better output? Why? Do you think these controls are enough?

In [ ]:
# Developer (system) prompt
system_prompt = """
You are an expert summarizer and reflective self-corrector.
You must faithfully summarize the provided document in a formal academic tone
while adhering to self-correction feedback from a prior evaluation.
Do NOT invent or insert information that does not exist in the text.
Focus on factual alignment, accuracy, coherence, and tone consistency.
Generate the final summary only — no analysis.
"""

# User prompt dynamically incorporating evaluation feedback
user_prompt = f"""
Here is the document context (excerpted from Peter F. Drucker’s 'Managing Oneself'):

{document_text[:6000]}

The previous summary evaluation metrics indicated:
- Summarization Score: 0.0 — added non-factual examples (self-awareness, longevity, Napoleon, etc.) absent in the source.
- Coherence Score: 0.76 — clear, logical flow.
- Tonality Score: 0.90 — strong academic tone but slightly verbose.
- Safety Score: 0.98 — safe and neutral language.

Write a corrected summary that:
1. Includes *only* verifiable information found within Drucker’s text.
2. Reflects his real arguments: the need for self-management, feedback analysis, awareness of strengths, values, and work style.
3. Removes all historical or imagined references unrelated to Drucker’s original prose.
4. Maintains formal academic tone and logical structure.
5. Remains concise yet complete (~200–250 words).
"""


from openai import OpenAI
client = OpenAI()

response = client.responses.create(
    model="gpt-4o",   # Not in GPT-5 family
    input=[
        {"role": "developer", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ],
)

improved_summary = response.output_text
print(improved_summary)



test_case = LLMTestCase(
    input=document_text,
    actual_output=improved_summary
)

# Step 6: Evaluate all metrics
evaluate_result = evaluate(
    test_cases=[test_case],
    metrics=[summarization_metric, coherence_metric, tonality_metric, safety_metric]
)

# Step 7 (corrected): Run .measure() manually for each metric
summarization_metric.measure(test_case)
coherence_metric.measure(test_case)
tonality_metric.measure(test_case)
safety_metric.measure(test_case)

# Step 8: Build valid structured results (all values now exist)
results = EvaluationResults(
    SummarizationScore=float(summarization_metric.score or 0.0),
    SummarizationReason=str(summarization_metric.reason or "No reason returned."),
    CoherenceScore=float(coherence_metric.score or 0.0),
    CoherenceReason=str(coherence_metric.reason or "No reason returned."),
    TonalityScore=float(tonality_metric.score or 0.0),
    TonalityReason=str(tonality_metric.reason or "No reason returned."),
    SafetyScore=float(safety_metric.score or 0.0),
    SafetyReason=str(safety_metric.reason or "No reason returned.")
)

print(results.model_dump_json(indent=2))



Please, do not forget to add your comments.


# Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

## Submission Parameters

- The Submission Due Date is indicated in the [readme](../README.md#schedule) file.
- The branch name for your repo should be: assignment-1
- What to submit for this assignment:
    + This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
- What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    + Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

## Checklist

+ Created a branch with the correct naming convention.
+ Ensured that the repository is public.
+ Reviewed the PR description guidelines and adhered to them.
+ Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.
